In [ ]:
!rm -rf sample_data

!wget http://dept-info.labri.fr/~hanna/ProjetClassif/features_adapte.csv
!wget http://dept-info.labri.fr/~hanna/ProjetClassif/features_head.csv
!wget http://dept-info.labri.fr/~hanna/ProjetClassif/train.csv
#Add test.csv ?


URL transformed to HTTPS due to an HSTS policy
--2022-12-12 19:28:34--  https://dept-info.labri.fr/~hanna/ProjetClassif/features_adapte.csv
Resolving dept-info.labri.fr (dept-info.labri.fr)... 147.210.9.83, 2001:660:6101:404::2:80
Connecting to dept-info.labri.fr (dept-info.labri.fr)|147.210.9.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 951108530 (907M) [text/csv]
Saving to: ‘features_adapte.csv’

features_adapte.csv 100%[===================>] 907.05M  28.5MB/s    in 32s     

2022-12-12 19:29:07 (28.3 MB/s) - ‘features_adapte.csv’ saved [951108530/951108530]

URL transformed to HTTPS due to an HSTS policy
--2022-12-12 19:29:07--  https://dept-info.labri.fr/~hanna/ProjetClassif/features_head.csv
Resolving dept-info.labri.fr (dept-info.labri.fr)... 147.210.9.83, 2001:660:6101:404::2:80
Connecting to dept-info.labri.fr (dept-info.labri.fr)|147.210.9.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19134 (19K) [text/csv]
Saving 

In [ ]:
import numpy as np
import pandas as pd

# Nom des features
features = pd.read_csv(filepath_or_buffer="features_head.csv", sep=",")
#print(features.columns)
#print(features)

# Jointure features/tracks du dataset train
traingenre = pd.read_csv(filepath_or_buffer="train.csv", sep=",")

iter_csv = pd.read_csv(filepath_or_buffer="features_adapte.csv", sep=",", iterator=True, chunksize=10000)
datatrain = pd.concat([chunk for chunk in iter_csv])

data = pd.merge(traingenre, datatrain, on='track_id')
print(data.shape, traingenre.shape, datatrain.shape)

# exemple
TRACK = 115
#print(datatrain.values[TRACK])

#training
xtrain = data.drop('genre_id',axis=1)
ytrain = data['genre_id'].values
#testing
print(xtrain.shape)
print(ytrain.shape)

(3995, 520) (3995, 2) (106574, 519)
(3995, 519)
(3995,)


In [ ]:
testgenre = pd.read_csv(filepath_or_buffer="test_clean.csv", sep=",")
data_test = pd.merge(testgenre, datatrain, on='track_id')
xtest = data_test
print(xtest.shape)


(4008, 519)


## **Catboost**

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 18.0 MB/s 


In [ ]:
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

In order to train and optimize our model, we need to utilize CatBoost library integrated tool for combining features and target variables into a train and test dataset. 

Our loss function:

In [ ]:
model = cb.CatBoostRegressor(
    loss_function = 'MAE',
    eval_metric = 'RMSE')

In [ ]:
model.fit(xtrain, ytrain)

0:	learn: 2.3212791	total: 241ms	remaining: 48s
1:	learn: 2.2926987	total: 371ms	remaining: 36.7s
2:	learn: 2.2615290	total: 507ms	remaining: 33.3s
3:	learn: 2.2345875	total: 640ms	remaining: 31.4s
4:	learn: 2.2099221	total: 778ms	remaining: 30.3s
5:	learn: 2.1890675	total: 924ms	remaining: 29.9s
6:	learn: 2.1669122	total: 1.07s	remaining: 29.5s
7:	learn: 2.1457414	total: 1.21s	remaining: 29.1s
8:	learn: 2.1238163	total: 1.36s	remaining: 28.9s
9:	learn: 2.1056727	total: 1.5s	remaining: 28.4s
10:	learn: 2.0852615	total: 1.65s	remaining: 28.3s
11:	learn: 2.0669159	total: 1.81s	remaining: 28.3s
12:	learn: 2.0469586	total: 1.96s	remaining: 28.2s
13:	learn: 2.0296248	total: 2.1s	remaining: 28s
14:	learn: 2.0136688	total: 2.25s	remaining: 27.8s
15:	learn: 1.9980529	total: 2.38s	remaining: 27.4s
16:	learn: 1.9842444	total: 2.53s	remaining: 27.2s
17:	learn: 1.9716256	total: 2.66s	remaining: 26.9s
18:	learn: 1.9575616	total: 2.8s	remaining: 26.7s
19:	learn: 1.9456889	total: 2.93s	remaining: 26.

In [ ]:
predicted_y = model.predict(xtest)
predictions = [round(value) for value in predicted_y]

In [ ]:
testgenre = np.array(testgenre)
predictions = np.array(predictions)
gn = []
for i in testgenre:
    gn.append(i[0])
print(len(gn))
print(predictions.shape)

dataset = pd.DataFrame({'track_id': gn, 'genre_id': predictions}, columns=['track_id', 'genre_id'])
print(dataset.head())
dataset.to_csv("res_catboost2.csv",index=False)

4008
(4008,)
   track_id  genre_id
0        10         7
1       141         4
2       190         4
3       193         4
4       194         4


## **xgboost**

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()
model.fit(xtrain, ytrain)

XGBClassifier(objective='multi:softprob')

In [ ]:
predicted_y = model.predict(xtest)
print(predicted_y.shape)
predictions = [round(value) for value in predicted_y]

(4008,)


In [ ]:
#accuracy = accuracy_score(y_test, predictions)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 61.87%


In [ ]:
testgenre = np.array(testgenre)
predictions = np.array(predictions)

gn = []
for i in testgenre:
    gn.append(i[0])
print(len(gn))
print(predictions.shape)

dataset = pd.DataFrame({'track_id': gn, 'genre_id': predictions}, columns=['track_id', 'genre_id'])
print(dataset.head())
dataset.to_csv("res_xgboost3.csv",index=False)

4008
(4008,)
   track_id  genre_id
0        10         8
1       141         3
2       190         3
3       193         3
4       194         3


## **lightgbm**

In [ ]:
import lightgbm as lgb

In [ ]:
clf = lgb.LGBMClassifier()
clf.fit(xtrain, ytrain)

LGBMClassifier()

In [ ]:
predicted_y=clf.predict(xtest)
predictions = [round(value) for value in predicted_y]

In [ ]:
#accuracy = accuracy_score(y_test, predictions)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 62.70%


In [ ]:
testgenre = np.array(testgenre)
predictions = np.array(predictions)

gn = []
for i in testgenre:
    gn.append(i[0])
print(len(gn))
print(predictions.shape)

dataset = pd.DataFrame({'track_id': gn, 'genre_id': predictions}, columns=['track_id', 'genre_id'])
print(dataset.head())
dataset.to_csv("res_lightgbm1.csv",index=False)

4008
(4008,)
   track_id  genre_id
0        10         8
1       141         3
2       190         3
3       193         3
4       194         3


## **Random Forest**

In [ ]:
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model=RandomForestClassifier(n_estimators=100)
model.fit(xtrain,ytrain)

RandomForestClassifier()

In [ ]:
predicted_y=model.predict(xtest)
predictions = [round(value) for value in predicted_y]

In [ ]:
testgenre = np.array(testgenre)
predictions = np.array(predictions)

gn = []
for i in testgenre:
    gn.append(i[0])
print(len(gn))
print(predictions.shape)

dataset = pd.DataFrame({'track_id': gn, 'genre_id': predictions}, columns=['track_id', 'genre_id'])
print(dataset.head())
dataset.to_csv("res_randomforest2.csv",index=False)

4008
(4008,)
   track_id  genre_id
0        10         8
1       141         3
2       190         5
3       193         3
4       194         3


## **Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
model = GaussianNB()
model.fit(xtrain, ytrain)

In [ ]:
predicted_y=model.predict(xtest)
predictions = [round(value) for value in predicted_y]

In [ ]:
testgenre = np.array(testgenre)
predictions = np.array(predictions)

gn = []
for i in testgenre:
    gn.append(i[0])
print(len(gn))
print(predictions.shape)

dataset = pd.DataFrame({'track_id': gn, 'genre_id': predictions}, columns=['track_id', 'genre_id'])
print(dataset.head())
dataset.to_csv("res_bayes1.csv",index=False)

4008
(4008,)
   track_id  genre_id
0        10         8
1       141         5
2       190         3
3       193         3
4       194         3


## **Decision tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier()
model.fit(xtrain,ytrain)

In [ ]:
predicted_y=model.predict(xtest)
predictions = [round(value) for value in predicted_y]

In [ ]:
testgenre = np.array(testgenre)
predictions = np.array(predictions)

gn = []
for i in testgenre:
    gn.append(i[0])
print(len(gn))
print(predictions.shape)

dataset = pd.DataFrame({'track_id': gn, 'genre_id': predictions}, columns=['track_id', 'genre_id'])
print(dataset.head())
dataset.to_csv("res_decisiontree.csv",index=False)

4008
(4008,)
   track_id  genre_id
0        10         8
1       141         7
2       190         1
3       193         5
4       194         8


## **SVM**

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
model = make_pipeline(StandardScaler(), SVC(gamma='auto'))
model.fit(xtrain,ytrain)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [ ]:
predicted_y=model.predict(xtest)
predictions = [round(value) for value in predicted_y]

In [ ]:
testgenre = np.array(testgenre)
predictions = np.array(predictions)

gn = []
for i in testgenre:
    gn.append(i[0])
print(len(gn))
print(predictions.shape)

dataset = pd.DataFrame({'track_id': gn, 'genre_id': predictions}, columns=['track_id', 'genre_id'])
print(dataset.head())
dataset.to_csv("res_svm1.csv",index=False)

4008
(4008,)
   track_id  genre_id
0        10         8
1       141         3
2       190         3
3       193         3
4       194         3
